# HW 1: Corporate Bond Pricing (due by 9.17 Tues)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2019`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2019/HW1/HW1.ipynb`__  (Please use the same name for repository and ipynb file)
* Add solution code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [3]:
import numpy as np

In [4]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [5]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(9.94512048770397, 10.0)

In [7]:
# Put your code here to price the corporate bond

def corp_bond(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4):
    
    U = np.random.uniform(size=n_sample)
    default_time = -(1/def_rate)*np.log(U)

    price_default_free = np.exp(-1 * rf_rate * mat)
    price1 = np.exp(-1 * rf_rate * default_time[default_time < mat]) * recovery
    price2 = price_default_free * np.ones_like(default_time[default_time >= mat])
    price = np.mean(np.hstack((price1, price2)))
    
    return price

# Call your function
price = corp_bond(mat, def_rate, rf_rate, recovery, n_sample)
print("The price of the bond using Monte-Carlo simulation is", price)

# Find the mean and std by calling the function 100 times. 
n_path = 100
price_lst0 = []
for i in range(n_path):
    price = corp_bond(mat, def_rate, rf_rate, recovery, n_sample)
    price_lst0.append(price)

print("price Mean and Std using Monte-Carlo simulation is", np.mean(price_lst0), np.std(price_lst0))


The price of the bond using Monte-Carlo simulation is 0.44086886037231543
price Mean and Std using Monte-Carlo simulation is 0.44049428001337526 0.0023016759538935947


### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [11]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=n_sample//2)
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(9.999999999999998, 10.0)

In [12]:
# Now include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4, antithetic=True, mean_match=True):

    if antithetic and not mean_match:
        U = np.random.uniform(size=n_sample//2)
        default_time = -(1/def_rate) * np.log(np.concatenate((U,1-U), axis=0))
        
    if mean_match and not antithetic:
        U = np.random.uniform(size=n_sample)
        default_time = -(1/def_rate) * np.log(U)
        default_time += 1/def_rate - default_time.mean()
        
    if mean_match and antithetic:
        U = np.random.uniform(size=n_sample//2)
        default_time = -(1/def_rate) * np.log(np.concatenate((U,1-U), axis=0))
        default_time += 1/def_rate - default_time.mean()
    
    price_default_free = np.exp(-1 * rf_rate * mat)
    price1 = np.exp(-1 * rf_rate * default_time[default_time < mat]) * recovery
    price2 = price_default_free * np.ones_like(default_time[default_time >= mat])
    price = np.mean(np.hstack((price1, price2)))
    
    return price

# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both
price_lst1, price_lst2, price_lst3  = [], [], []
for i in range(n_path):
    price1 = corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample, antithetic=True, mean_match=False)
    price2 = corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample, antithetic=False, mean_match=True)
    price3 = corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample, antithetic=True, mean_match=True)
    price_lst1.append(price1)
    price_lst2.append(price2)
    price_lst3.append(price3)

print("price Mean and Std using antithetic method is", np.mean(price_lst1), np.std(price_lst1))
print("price Mean and Std using mean_match method is", np.mean(price_lst2), np.std(price_lst2))
print("price Mean and Std using antithetic and mean_match method is", np.mean(price_lst3), np.std(price_lst3))

price Mean and Std using antithetic method is 0.440670045061649 0.001842184736807297
price Mean and Std using mean_match method is 0.44007874954475507 0.0015187097182368472
price Mean and Std using antithetic and mean_match method is 0.44043913541700974 0.0015644966424027927


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [13]:
### Put the analytic expression for the corporate bond price
def pricing_analytic(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3):

    exp1 = np.exp(-1 * (rf_rate + def_rate) * mat)
    price = exp1 + (1-exp1) * def_rate * recovery / (def_rate+rf_rate)

    return price

price = pricing_analytic(mat, def_rate, rf_rate, recovery)
print("The analytic value of the corporate bond is", price)

The analytic value of the corporate bond is 0.44040907156462505


In [14]:
print("The error between analytic value and Monte-Carlo simulation is", price - np.mean(price_lst0))
print("The error between analytic value and antithetic method is", price - np.mean(price_lst1))
print("The error between analytic value and mean-match method is", price - np.mean(price_lst2))
print("The error between analytic value and antithetic & mean-match method is", price - np.mean(price_lst3))

The error between analytic value and Monte-Carlo simulation is -8.520844875020694e-05
The error between analytic value and antithetic method is -0.0002609734970239197
The error between analytic value and mean-match method is 0.00033032201986998855
The error between analytic value and antithetic & mean-match method is -3.006385238468745e-05
